# Topic Modeling Tutorial
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [68]:
# !pip install gensim==3.8.3

In [69]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# NTLK stopwords
from nltk.corpus import stopwords

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import pickle 

import warnings
warnings.filterwarnings("ignore")

In [70]:
#update stopwords 
stop_words = stopwords.words('english')
stop_words.extend(['use', 'used', 'using', 'one', 'like', 'user', 'make', 'also', 'get', 'point', 'let', 'go'])

In [71]:
filename = 'data/cleaned_no_outliers.csv'
output_filename = 'data/spacy_preprocess.csv'

In [72]:
df = pd.read_csv(filename)

In [73]:
#remove non-text, non-Y columns
df = df.drop(columns=['date', 'responses', 'author_url',
       'story_url', 'reading_time (mins)', 'number_sections', 
       'number_paragraphs'])

#### Combine Title & Subtitle

In [74]:
df['combined_title'] = (df['title'] + ' ' + df['subtitle'])

In [75]:
df = df.drop(columns = ['title', 'subtitle']).reset_index(drop=True)

#### Remove Punctuation & Make Lowercase

In [76]:
text_cols = ['section_titles', 'paragraphs', 'combined_title']

In [77]:
for col in text_cols:
    df[col] = df[col].map(lambda x: re.sub("[''·,\.!?'-]", '', x).lower())

#### Remove the 'Min Read' Phrase at the Beginning of Each Paragraph

In [78]:
#remove all text up to and including 'min read' in the beginning of the paragraphs
df['paragraphs'] = df['paragraphs'].map(lambda x: (x[x.index('min read')+9:] if (x.count('min read') > 0) else x))
                     

#### Remove the Brackets Surrounding The Section Titles

In [79]:
#remove brackets surrounding the section titles
df['section_titles'] = df['section_titles'].map(lambda x: x.replace("[",""))
df['section_titles'] = df['section_titles'].map(lambda x: x.replace("]",""))

#### Combine Text Columns

In [80]:
df['text'] = df['combined_title'] + df['section_titles'] + df['paragraphs']

In [81]:
df = df.drop(columns = ['section_titles', 'paragraphs', 'combined_title'])

In [82]:
# Convert to list
data = df['text'].values.tolist()

#### Tokenize Words and Clean Up Text

In [83]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

data_words = list(sent_to_words(data))

#### Bigram and Trigram Models

In [84]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [85]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#### Remove Stop Words 

In [86]:
data_words_nostops = remove_stopwords(data_words)

#### Form Bigrams

In [87]:
data_words_bigrams = make_bigrams(data_words_nostops)

#### Initialize Spacy 'en' Model

In [88]:
#!spacy download en_core_web_lg

In [89]:
nlp = spacy.load('en_core_web_lg')

#### Lemmatize

In [90]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV','PROPN'])

In [ ]:
type(data_lemmatized)

#### Create the Dictionary & Corpus Needed for Topic Modeling

In [91]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [92]:
#check out the dictionary

id2word[0]

'abbreviate'

In [41]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abbreviate', 1),
  ('ability', 1),
  ('able', 2),
  ('ace', 5),
  ('achieve', 1),
  ('actual', 1),
  ('adapt', 1),
  ('addition', 1),
  ('additional', 1),
  ('aim', 1),
  ('airbnb', 1),
  ('algorithm', 5),
  ('allow', 1),
  ('always', 1),
  ('amazon', 1),
  ('amount', 1),
  ('analysis', 1),
  ('analytic', 1),
  ('answer', 3),
  ('appear', 5),
  ('apply', 4),
  ('approach', 3),
  ('approximate', 1),
  ('article', 1),
  ('ask', 7),
  ('aspect', 1),
  ('assumption', 2),
  ('base', 2),
  ('basic', 5),
  ('begin', 1),
  ('benefit', 1),
  ('best_practice', 1),
  ('book', 1),
  ('breadthfirst_search', 1),
  ('business', 3),
  ('candidate', 9),
  ('capability', 1),
  ('career', 1),
  ('carry', 1),
  ('catch', 1),
  ('categorization', 1),
  ('categorize', 3),
  ('category', 3),
  ('chance', 1),
  ('check', 2),
  ('choice', 3),
  ('choose', 1),
  ('clarify', 1),
  ('classic', 1),
  ('clearly', 2),
  ('code', 45),
  ('coderpad', 1),
  ('codesignal', 1),
  ('coding', 7),
  ('collection', 1),
 

### Build the LDA Model

In [47]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [48]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.015*"system" + 0.014*"ai" + 0.009*"model" + 0.009*"machine" + '
  '0.008*"application" + 0.007*"ml" + 0.007*"process" + 0.006*"technology" + '
  '0.006*"human" + 0.006*"design"'),
 (1,
  '0.048*"image" + 0.033*"model" + 0.019*"training" + 0.018*"network" + '
  '0.017*"layer" + 0.015*"deep" + 0.014*"learning" + 0.014*"train" + '
  '0.013*"input" + 0.011*"learn"'),
 (2,
  '0.012*"time" + 0.010*"cluster" + 0.010*"analysis" + 0.009*"number" + '
  '0.008*"see" + 0.008*"show" + 0.008*"customer" + 0.008*"datum" + '
  '0.007*"group" + 0.007*"high"'),
 (3,
  '0.030*"datum" + 0.014*"science" + 0.013*"work" + 0.012*"data" + '
  '0.012*"learn" + 0.008*"time" + 0.008*"scientist" + 0.007*"good" + '
  '0.007*"project" + 0.007*"well"'),
 (4,
  '0.086*"datum" + 0.032*"data" + 0.028*"column" + 0.019*"table" + '
  '0.014*"value" + 0.014*"dataframe" + 0.013*"panda" + 0.013*"query" + '
  '0.012*"row" + 0.012*"dataset"'),
 (5,
  '0.101*"model" + 0.032*"feature" + 0.030*"datum" + 0.025*"dataset" + 

### Compute Model Perplexity and Coherence Score

In [49]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.26654086129321

Coherence Score:  0.4660213476164413


### Visualize

In [50]:
pyLDAvis.enable_notebook()
plot = gensimvis.prepare(lda_model, corpus, id2word)
# Save pyLDA plot as html file
pyLDAvis.save_html(plot, 'figures/LDA_spacy_TDS10.html')

In [51]:
plot

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.030207  0.123446       1        1  22.375451
6     -0.056686  0.152283       2        1  12.344020
7     -0.071640 -0.037454       3        1  10.948212
8     -0.159451 -0.002709       4        1  10.504419
2     -0.029455  0.064802       5        1   9.496284
0      0.271836  0.131546       6        1   8.609148
5      0.039156 -0.193645       7        1   8.037480
1      0.143310 -0.176809       8        1   6.780344
4     -0.117156  0.033140       9        1   6.564469
9     -0.050121 -0.094599      10        1   4.340172, topic_info=          Term          Freq         Total Category  logprob  loglift
241      model  60084.000000  60084.000000  Default  30.0000  30.0000
81       datum  87347.000000  87347.000000  Default  29.0000  29.0000
932      image  18667.000000  18667.000000  Default  28.0000  28.0000
80        data  28940.000000  28940.000000  Default  27.0000  27.0000
926   function  25168.000000  25168.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
149       give   1160.572340  14576.312220  Topic10  -5.2778   0.6068
241      model   1292.413333  60084.671598  Topic10  -5.1702  -0.7020
16    analysis   1097.717887  10585.627371  Topic10  -5.3334   0.8710
753     result   1070.525921  16583.794466  Topic10  -5.3585   0.3970
1003      step   1057.744059  13582.224921  Topic10  -5.3705   0.5846

[745 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
10739      6  0.996302  acceleration
18101     10  0.997201        accent
1540       1  0.029747        access
1540       2  0.601013        access
1540       5  0.015083        access
...      ...       ...           ...
413        1  0.505629          year
413        5  0.417726          year
413        6  0.035334          year
413        9  0.041366          year
13006      8  0.995847          yolo

[1885 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 8, 9, 3, 1, 6, 2, 5, 10])

#### Build LDA Mallet Model

In [55]:
import os
os.environ.update({'/Users/rachelinsler/dsi/downloads/mallet-2.0.8/bin/':r'Python/Mallet/mallet-2.0.8/bin'})

In [97]:
from gensim.models.wrappers import LdaMallet
import gensim.models.word2vec 

ImportError: cannot import name 'Vocab' from 'gensim.models.word2vec' (/Users/rachelinsler/ENTER/envs/capstone/lib/python3.9/site-packages/gensim/models/word2vec.py)

In [ ]:
mallet_path = '/Users/rachelinsler/dsi/downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
df.to_csv(output_filename, index=False)